# PC1 : the theory of thermal explosion

according to Semenov and Frank-Kamenetskii

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Model 1 - adiabatic explosion

**Graph of the function $ T \rightarrow \Lambda(T) = B e^{-\beta T_0/T} (T_b - T)$**

In [ ]:
T0 = 400.
Tb = 3000.
T = np.linspace(T0,Tb,1000)

beta = np.arange(30, 101, 10)

fig = go.Figure()

for i, beta_i in enumerate(beta):
    #print(beta_i)
    Lambda = np.exp(-(beta_i*T0)/T) * (Tb-T)
    fig.add_trace(go.Scatter(visible=False, x=T, y=Lambda))

fig.data[0].visible = True

# create slider
steps = []
for i, beta_i in enumerate(beta):
    step = dict(method="update", label = f"{beta_i}", args=[{"visible": [(el==i) for el in range(beta.size)]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'beta = '}, steps=steps)]

fig.update_layout(sliders=sliders, yaxis = dict(exponentformat = 'e'), title="Lamba(𝑇)")
fig.show()

**Evolution of $\widetilde{\theta}$ solution of $\, d_{\tau} \widetilde{\theta} = e^{\widetilde{\theta}}$**

In [ ]:
def fcn(t, theta): 
    return np.exp(theta)

tini = 0.0
tend = 1.0

yini = (0.0,)

sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', rtol=1.e-12, atol=1.e-12)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0]))
fig.update_xaxes(range=[0, 1.1])
fig.update_layout(title="Evolution of theta(t)")
fig.show()

## Model 2 - explosion taking into account thermal losses

$$
\left\{\begin{aligned}
d_{\tau} \widetilde{\theta} & = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \widetilde{Y} - \alpha_0 \, \widetilde{\theta} \\
d_{\tau} \widetilde{Y} & = -\frac{1}{\overline T_r} \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg)  \widetilde{Y}
\end{aligned}\right.
$$

In [ ]:
class second_model:

    def __init__(self, alpha, Tr, beta):
        self.alpha = alpha
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, y):
        theta, Y = y
        alpha = self.alpha
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * Y - alpha*theta 
        Y_dot     = (-1/Tr) * np.exp(theta/(1+(theta/beta))) * Y
        return (theta_dot, Y_dot)

**Evolution of $\widetilde{\theta}$ and $Y(t)$**

In [ ]:
alpha = np.arange(0, 2.7, 0.2)
Tr = 200.0
beta = 30.

tini = 0.0
tend = 10.0

yini = (0.0, 1.0)

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.1,
                    subplot_titles=("Evolution of theta(t)", "Evolution of Y(t)"))

for i, alpha_i in enumerate(alpha):
    sm = second_model(alpha_i, Tr, beta)
    fcn = sm.fcn
    sol = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    fig.add_trace(go.Scatter(visible=False, x=sol.t, y=sol.y[0], line_color='#636EFA'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol.t, y=sol.y[1], line_color='#636EFA'), row=2, col=1)

fig.data[0].visible = True
fig.data[1].visible = True

# create slider
steps = []
for i, alpha_i in enumerate(alpha):
    step = dict(method="update", label = f"{alpha_i:.1f}", 
                args=[{"visible": [(el==2*i) or (el==2*i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]
    
fig.update_layout(sliders=sliders, showlegend=False, height=800, width=800)
fig.show()

**Graphs of  $e^\widetilde{\theta}$ and $q^- (\widetilde{\theta})= \alpha_0 \, \widetilde{\theta}$**

In [ ]:
e = np.exp(1)
eover10 = np.exp(1) / 10
alpha = np.arange(e-5*eover10, e+5*eover10, eover10)

theta = np.linspace(0, 2.5,1000)

exp = np.exp(theta)
q = alpha_i*theta

fig = go.Figure()
fig.add_trace(go.Scatter(x=theta, y=exp, name="exp(theta)"))

for i, alpha_i in enumerate(alpha):
    q = alpha_i*theta
    fig.add_trace(go.Scatter(visible=False, x=theta, y=q, name="q"))
    
fig.data[1].visible = True

steps = []
for i, alpha_i in enumerate(alpha):
    step = dict(method="update", label = f"{alpha_i:.3f}", 
                args=[{"visible": [(el==0) or (el==i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha0 = '}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.show()

## Model 3 - explosion taking into account convection

$$
\left\{\begin{aligned}
d_{\tau} \widetilde{\theta} & = e^{\widetilde{\theta}} - \alpha_0 \, (1 + \mu \, \psi^2) \, \widetilde{\theta} \\
d_{\tau} \psi & = -a \, \psi (\psi^2 + \widetilde{\theta_c} - \widetilde{\theta})
\end{aligned}\right.
$$

In [ ]:
class third_model:

    def __init__(self, alpha0, mu, a, thetac):
        self.alpha0 = alpha0
        self.mu = mu
        self.a = a
        self.thetac = thetac

    def fcn(self, t, y):
        theta, psi = y
        alpha0 = self.alpha0
        mu = self.mu
        a = self.a
        thetac = self.thetac
        theta_dot = np.exp(theta) - alpha0*(1 + mu*psi*psi) * theta
        psi_dot = -a*psi*(psi*psi+thetac-theta)
        return (theta_dot, psi_dot)

**Evolution of $\widetilde{\theta}(t)$ and phase plan $(\widetilde{\theta}(t), \psi)$**

In [ ]:
alpha0 = 2.7 
mu = 0.7
a =  4.3

thetac = np.arange(1.0, 1.066, 0.005)

tini = 0. 
tend = 300.

yini = (1.0, 0.4)

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.1,
                    subplot_titles=("Evolution of theta(t)", "Phase plan (theta, psi)"))

for i, thetac_i in enumerate(thetac):
    tm = third_model(alpha0, mu, a, thetac_i)
    fcn = tm.fcn
    sol = solve_ivp(fcn, (tini, tend), yini, method='RK45', atol=1.e-8, rtol=1.e-8)
    fig.add_trace(go.Scatter(visible=False, x=sol.t, y=sol.y[0], line_color='#636EFA'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol.y[0], y=sol.y[1], line_color='#636EFA'), row=2, col=1)

fig.data[0].visible = True
fig.data[1].visible = True

# create slider
steps = []
for i, thetac_i in enumerate(thetac):
    step = dict(method="update", label = f"{thetac_i:.3f}", 
                args=[{"visible": [(el==2*i) or (el==2*i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'thetac = '}, steps=steps)]

fig.update_layout(sliders=sliders, showlegend=False, height=800, width=800)
fig.show()